In [20]:
import datetime

now = datetime.datetime.now()
now = str(now)
SYMBOL = 'APPL'
search_term = 'apple'

In [21]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

alpha_vantage_APIKEY = '4FSNXUSUS1Y22Q4A'

ts = TimeSeries(key=alpha_vantage_APIKEY, output_format='pandas')
data, meta_data = ts.get_intraday(symbol=SYMBOL,interval='60min', outputsize='full')
data['4. close'].plot()
plt.title('Intraday Times Series for the MSFT stock (1 min)')
plt.show()

KeyError: 'Time Series (60min)'

In [ ]:
#rename the columns to something more sensible
data.rename(columns={'1. open':'Open',
                  '2. high':'High',
                  '3. low':'Low',
                  '4. close':'Close',
                  '5. volume':'Volume',
                }, inplace=True)

#get end of day only
cyclic_data = data.iloc[6::7, :]
#subtract the difference
cyclic_data.Close = cyclic_data.Close.diff(periods=-1)
#drop the latest value (ie. there's no tomorrow minus today)
cyclic_data = cyclic_data[:-1]

# Scraper


In [22]:
from urllib.request import urlopen, HTTPError
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint

date_sentiments = {}

for i in range(1,11):
#     page = urlopen('https://www.businesstimes.com.sg/search/crude%2Boil?page='+str(i)).read()
    page = urlopen('https://www.businesstimes.com.sg/search/{}?page='.format(search_term) + str(i)).read()
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})
    for post in posts:
        time.sleep(1)
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            link_page = urlopen(url).read()
        except HTTPError as e:
            if e.code == 403:
                continue
            url = url[:-2]
            link_page = urlopen(url).read()
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        date_sentiments.setdefault(date, []).append(passage)

12 Apr 2019 https://www.businesstimes.com.sg/technology/apple-adds-more-suppliers-to-clean-energy-programme
11 Apr 2019 https://www.businesstimes.com.sg/technology/apple-adds-foxconn-chip-suppliers-to-clean-energy-programme
09 Apr 2019 https://www.businesstimes.com.sg/companies-markets/banks-should-worry-about-apple-card
06 Apr 2019 https://www.businesstimes.com.sg/technology/does-apple-want-to-sell-tv-subscriptions-or-change-the-world
04 Apr 2019 https://www.businesstimes.com.sg/technology/japan-display-to-supply-oled-screens-for-apple-watch
02 Apr 2019 https://www.businesstimes.com.sg/energy-commodities/saudi-aramco-is-world%E2%80%99s-most-profitable-company-beating-apple-by-far
02 Apr 2019 https://www.businesstimes.com.sg/technology/apple-supplier-japan-display-to-seek-funding-shares-surge
01 Apr 2019 https://www.businesstimes.com.sg/technology/troubled-apple-supplier-japan-display-to-seek-funding-shares-surge
30 Mar 2019 https://www.businesstimes.com.sg/consumer/apple-pulls-plug-on

04 Jan 2019 https://www.businesstimes.com.sg/government-economy/apples-cut-in-revenue-outlook-triggers-slump-for-suppliers
04 Jan 2019 https://www.businesstimes.com.sg/companies-markets/a-bite-of-apples-poison-and-markets-sell-in-a-flash
04 Jan 2019 https://www.businesstimes.com.sg/government-economy/apples-china-trouble-makes-trumps-trade-war-harder-to-defend
03 Jan 2019 https://www.businesstimes.com.sg/stocks/us-wall-st-opens-lower-as-apple-sales-warning-drags-tech
03 Jan 2019 https://www.businesstimes.com.sg/stocks/europe-shares-fall-as-apple-guidance-cut-whipsaws-tech-stocks
03 Jan 2019 https://www.businesstimes.com.sg/stocks/asia-markets-stage-slight-recovery-but-tech-firms-soured-by-apple
03 Jan 2019 https://www.businesstimes.com.sg/government-economy/apple-isn%E2%80%99t-the-only-casualty%C2%A0of-chinas-slowdown
03 Jan 2019 https://www.businesstimes.com.sg/stocks/asia-shares-skid-as-apple-warning-stokes-growth-fears-flash-crash-sweeps-currencies
03 Jan 2019 https://www.businessti

In [ ]:
import json

json.dump(date_sentiments, open( 'businesstimes_{}_{}.json'.format(SYMBOL, now), 'w' ))

In [ ]:
date_sentiment = {}

def flatten(x):
    result = []
    for el in x:
        if hasattr(el, "__iter__") and not isinstance(el, (str, bytes)):
            result.extend(flatten(el))
        else:
            result.append(el)
    return result

for k,v in date_sentiments.items():
    temp = flatten(v)
    seperator = ', '
    temp = seperator.join(temp)
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = temp

In [ ]:
earliest_date = min(date_sentiment.keys())
latest_date = max(date_sentiment.keys())

# Merge Data

In [ ]:
import pandas as pd

train_df = pd.DataFrame.from_dict(date_sentiment.items())

In [ ]:
new_frame = cyclic_data.reset_index()

In [ ]:
new_frame.head()

In [ ]:
new_frame['date'] = pd.to_datetime(new_frame['date']).dt.date

In [ ]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

shared_dates = intersection(train_df[0], new_frame['date'])

In [ ]:
train_df = train_df[train_df[0].isin(shared_dates)]

In [ ]:
def set_intervals(item):
    if item > 0:
        return 1
    elif item < 0:
        return 0
new_frame.Close.diff(periods=-1)
new_frame['diff'] = new_frame['Close'].map(set_intervals)

In [ ]:
new_frame = new_frame[new_frame['date'].isin(shared_dates)]

In [ ]:
compiled_frame = train_df[::-1]
compiled_frame['target'] = new_frame['diff'].values
compiled_frame.target.astype(int)
compiled_frame

In [ ]:
len(compiled_frame)

In [ ]:
file_name = 'data/mycollection/{}_{}'.format(SYMBOL, now)
compiled_frame.to_csv(file_name, sep='\t')

In [ ]:
pd.read_csv(file_name, sep='\t')